### CS2101 - Programming for Science and Finance
Prof. Götz Pfeiffer<br />
School of Mathematical and Statistical Sciences<br />
University of Galway

# Computer Lab 7

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from IPython.display import display, Markdown

## 1. Numerical Differentiation.

1. As the derivative of a function $f \colon D \to \mathbb{R}$ (for some $D \subseteq \mathbb{R}$) at a point $x \in D$ is given by the limit 
  $$
  f'(x) = \lim_{h \to 0} \frac{f(x+h) - f(x-h)}{2h}
  $$
  we can use
  $$
  f'(x) \approx \frac{f(x+h) - f(x-h)}{2h}
  $$
  for small values of $h$ as an approximation of the value of the derivative of $f$ at $x$.  Write a function `derivative` that takes a function $f$ and a small value $h$ as input and the computes and returns the derivative $f'$ as a function, using the above approximation.  Can you use a `lambda` expression to avoid having to invent a variable name for the derivative?

In [ ]:
def derivative(f, h):
    return lambda x: (f(x + h) - f(x - h)) / (2 * h)


2. Check, perhaps with $h = 10^{-5}$, that your function computes $-\sin(x)$ as the derivative of $\cos(x)$ by plotting both your derivative and the $-\sin(x)$ function over the interval $[0, 2\pi]$.

In [ ]:
x_values = np.linspace(0, 2 * math.pi, 100)

f = math.cos
h = 1E-5
f_prime = derivative(f, h)

y_values = [f_prime(x) for x in x_values]
y_actual = [-math.sin(x) for x in x_values]

plt.figure(figsize=(10, 6))
plt.plot(x_values, y_values, label="Approximate derivative of cos(x)", linestyle='--')
plt.plot(x_values, y_actual, label="-sin(x)", alpha=0.5)
plt.title("Comparison of Approximate Derivative of cos(x) and -sin(x)")
pi_ticks = [0, math.pi/2, math.pi, 3*math.pi/2, 2*math.pi]
pi_labels = ['0', 'π/2', 'π', '3π/2', '2π']
plt.xticks(pi_ticks, pi_labels)
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid(True)
plt.show()




3. Using your `derivative` function, plot $f(x) = 3x^4 - 4x^3 - 12x^2 + 5$ (over the interval $[-2, 3]$), its first derivative $f'$ (over the interval $[-1.4,2.3]$) and its second derivative $f''$ (over the interval $[-1,1.6]$).

In [ ]:
def f(x):
    return (3*x**4) - (4*x**3) - (12*x**2) + 5

h = 1E-5
f_prime = derivative(f, h)
f_double_prime = derivative(f_prime, h)

# x values
x_f = np.linspace(-2, 3, 100)
x_f_prime = np.linspace(-1.4, 2.3, 100)
x_f_double_prime = np.linspace(-1, 1.6, 100)

# Calculate y values
y_f = [f(x) for x in x_f]
y_f_prime = [f_prime(x) for x in x_f_prime]
y_f_double_prime = [f_double_prime(x) for x in x_f_double_prime]

plt.figure(figsize=(10, 6))
plt.plot(x_f, y_f, label="$f(x) = 3x^4 - 4x^3 - 12x^2 + 5$")
plt.plot(x_f_prime, y_f_prime, label="$f'(x)$")
plt.plot(x_f_double_prime, y_f_double_prime, label="$f''(x)$")
plt.legend()
plt.grid(True)
plt.show()

# Plot f(x)
plt.figure(figsize=(10, 6))
plt.plot(x_f, y_f, label="$f(x) = 3x^4 - 4x^3 - 12x^2 + 5$")
plt.title("Function $f(x)$")
plt.xlabel("x")
plt.ylabel("$f(x)$")
plt.legend()
plt.grid(True)
plt.show()

# Plot f'(x)
plt.figure(figsize=(10, 6))
plt.plot(x_f_prime, y_f_prime, label="$f'(x)$", color='orange')
plt.title("First Derivative $f'(x)$")
plt.xlabel("x")
plt.ylabel("$f'(x)$")
plt.legend()
plt.grid(True)
plt.show()

# Plot f''(x)
plt.figure(figsize=(10, 6))
plt.plot(x_f_double_prime, y_f_double_prime, label="$f''(x)$", color='red')
plt.title("Second Derivative $f''(x)$")
plt.xlabel("x")
plt.ylabel("$f''(x)$")
plt.legend()
plt.grid(True)
plt.show()


## 2.  Newton's Method

1.  Using your function `derivative`, implement Newton's method `newton` from the lectures.

In [ ]:
def newton(f, eps, h):
    fprime = derivative(f,h)
    def solve(x):
        while abs(f(x)) >= eps:
            x -= f(x)/fprime(x)
        return x
    return solve

2.  Use your implementation of Newton's method to find solutions to the equation $\cos(x) = \sqrt{2}/2$, say for $50$ different starting values $x_0$ in the interval $[1,11]$. Check that all your solutions are integer multiples of $\pi/4$.  Modulo $8$, which integers do occur? (And which ones don't?)

In [ ]:
eps = h =  1E-5
f = lambda x: math.cos(x) - math.sqrt(2)/2
x_0 = np.linspace(1, 11, 50)

eq_solve = newton(f, eps, h)
approx_roots = np.array([eq_solve(x) for x in x_0])

# Convert solutions to integers
multiples = approx_roots * 4 / math.pi
numbers = np.round(multiples, 0)

# Calculate modulo 8
mod_8_set = set(numbers % 8)
missing_mod_8 = set(range(8)) - mod_8_set

In [ ]:
# Display solutions
display(Markdown(
        rf"""Solutions that are integer multiples of $\frac{{\pi}}{{4}} \mod 8$:
        {sorted(int(sol) for sol in mod_8_set)}"""))

display(Markdown(
    fr"Integers $\text{{mod}}\; 8$ that do not occur: {sorted(missing_mod_8)}"
))

## 3.  Solving Differential Equations

1. [Euler's method](https://en.wikipedia.org/wiki/Euler_method) is a simple method for solving a first-order initial value problem of the form
  $$
  \begin{cases}
  y' = f(t, y(t)) \\
  y(t_0) = y_0
  \end{cases}
  $$
  numerically.  Using the approximation
  $$
  y'(t_0) = \frac{y(t_0 + h) - y(t_0)}{h}
  $$
  for the derivative of $y$ at a point $t_0$, one obtains for any chosen step-size $h$ a solution
  $$
  y_1 = y(t_0+h) = y(t_0) + h f(t_0, y(t_0)) = y_0 + h f(t_0, y_0).
  $$
  This process can be iterated.  Letting $t_n = t_0 + nh$, we compute $y_n$, an approximation of $y(t_n)$ by iterating the following step:
  $$
  y_{i+1} = y_i + h f(t_i, y_i).
  $$
  Implement this process as a function `euler` as follows.  The function `euler` has $4$ parameters: the function $f$, the initial value $t_0$, the initial value $y_0$ and the step-size $h$.  From these it constructs an returns a function `approx` so that a call `approx(n)`  computes and returns the approximation $y_n$ of $y(t_n)$.

In [ ]:
def euler(f, t0, y0, h):
    def approx(n):
        # Initialize variables
        t = t0
        y = y0
        # Iterate to approximate the solution
        for _ in range(n):
            # Compute the next y and t using Euler's method
            y = y + h * f(t, y)
            t = t + h
        return y
    
    return approx

2. Use your function `euler` to approximately solve the initial value problem
  $$
  \begin{cases}
  y' = 2y + t^2 - t \\
  y(0) = 1
  \end{cases}
  $$
  using a step size of $h = 0.01$. (This initial-value problem has the exact solution $y(t) = e^{2t} -t^2/2$ which can be verified by differentiation.)

In [ ]:
# Define the function f(t,y) = 2y + t^2 - t
def f(t,y):
    return 2*y + t**2 -t

# Exact solution for comparison
def exact_solution(t):
    return np.exp(2 * t) - t**2 / 2

# Initial conditions
t0 = 0
y0 = 1
h = 0.01
steps = int(1 / h)  # Number of steps to reach t=1
t_values = np.linspace(0, 1, steps + 1)

# Compute approximate and exact solutions
approx = euler(f, t0, y0, h)
euler_results = [approx(i) for i in range(steps + 1)]
exact_results = [exact_solution(t) for t in t_values]

3. Plot your solution and the exact solution over `np.linspace(0, 1, 101)`.

In [ ]:
# Adjusting to plot solutions over np.linspace(0, 1, 101)
t_values_fixed = np.linspace(0, 1, 101)

# Recalculating Euler results over the fixed t_values
step_size_fixed = 0.01
approx_fixed = euler(f, t0, y0, step_size_fixed)
euler_results_fixed = [approx(int(t / step_size_fixed)) for t in t_values_fixed]
exact_results_fixed = [exact_solution(t) for t in t_values_fixed]

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(t_values_fixed, euler_results_fixed, label="Euler Approximation", linestyle="--")
plt.plot(t_values_fixed, exact_results_fixed, label="Exact Solution", linestyle="-")
plt.xlabel("t")
plt.ylabel("y")
plt.title("Euler Approximation vs Exact Solution (Refined t-values)")
plt.legend()
plt.grid(True)
plt.show()


## 4.  Numerical Integration

* One well-known method to approximate a definite integral
  $$
  I = \int_a^b f(x) \, dx.
  $$
  is the [Trapezoid method](https://en.wikipedia.org/wiki/Trapezoidal_rule). It approximates the definite integral $I$ by the area beneath the graph of $f$ by trapezoids.  For this, we choose a value for the parameter $N$, which is the number of subintervals we divide the interval $[a,b]$ into. Then we get the approximation
  $$
  I \approx  \frac{b-a}{2N}(f(x_0) + 2f(x_1) + 2 f(x_3) + \cdots + 2f(x_{N-1}) + f(x_N)).
  $$

1. Your task is to write a function `trapezoid`, taking two parameters: the function `f` to integrate, and the number `N` of subintervals.  It then constructs and returns a function `F`, say, of two parameters `a` and `b`,  which approximates $I$ according to the above rule. Thus, for example:
   ```python
       F = trapezoid(lambda x: x, 10)
       F(1,2)
   ```
   should yield the value $1.5$.

In [ ]:
def trapezoid(f, N):
    def F(a, b):
        # Calculate the width of each subinterval
        h = (b - a) / N
        # Compute the x-values for the interval
        x_values = [a + i * h for i in range(N + 1)]
        # Compute the sum of the function values
        integral = f(x_values[0]) + f(x_values[-1])  # f(x_0) + f(x_N)
        integral += 2 * sum(f(x) for x in x_values[1:-1])  # 2 * sum of interior points
        # Multiply by the width of the trapezoids
        return (h / 2) * integral

    return F

In [ ]:
F = trapezoid(lambda x : x, 10)
F(1,2)

2.  Approximate $I = \int_0^1 e^{x^2}\, dx$, using your function `trapezoid` once with $10$ subintervals, then with $100$ subintervals, and again with $1000$ subintervals.

In [ ]:
# Define the function to integrate
f = lambda x: np.exp(x**2)

# Create the trapezoid approximation function for different subintervals
F_10 = trapezoid(f, 10)
F_100 = trapezoid(f, 100)
F_1000 = trapezoid(f, 1000)

# Compute the integral over [0, 1] with the different approximations
result_10 = F_10(0, 1)
result_100 = F_100(0, 1)
result_1000 = F_1000(0, 1)

print(f"10 Subintervals: {result_10}")
print(f"100 Subintervals: {result_100}")
print(f"1000 Subintervals: {result_1000}")


## 5.  Decorators

* A **decorator** is a function that takes a function `f` as argument, constructs and returns a new function `ff` which is then reassigned to `f`.  In Python, a decorator can be applied to function definition with the `@` syntax.

1. Define a decorator `logCall` that, when applied to a function `f`, each time `f` is called, the function call with its arguments and the return value are printed as in `f(x, y) = z`.  For example, if `f` computes the absolute value of the difference of `x` and `y`, then after defining
   ```python
      @logCall
      def f(x, y):
          return abs(x - y)
   ```
   a call like `f(3, 5)` should cause a printed message like
   ```python
      f(3, 5) = 2
   ```

In [ ]:
def logCall(f):
    def ff(*args):
        result = f(*args)
        print(f"{f.__name__}({', '.join(map(str, args))}) = {result}")
        return result
    return ff

2.  Test your decorator on the function `f(x, y)` above.

In [ ]:
@logCall
def f(x, y):
    return abs(x - y)

In [ ]:
f(3, 5)

3. Define a version of the greatest common denominator, decorated with `@logCall`.  Then test `gcd(60, 24)`.

In [ ]:
@logCall
def gcd(a, b):
    return a if b == 0 else gcd(b, a % b)

In [ ]:
gcd(60,24)

4. A decorator need not be a function, it only has to be **callable**.  An object `o` of a class `C` is callable, if the class `C` implements the special method `__call__`.  Then a function call like `o(arg)` will refer back to `o.__call__(arg)`.  Write a **decorator class** `countedFunc` which for the decorated function counts how often it gets called.  This class should implement two special functions: `__init__` has parameters `self` and the function `func` to be decorated, and should assign `func` to the attribute `func` of `self`, and initialize an attribute `count` of `self` as $0$; and `__call__` should have arguments `self` and `*args`, increment the counter `self.count` by one, and then call the function `self.func` with arguments `*args` and return the resulting value.

In [ ]:
class countedFunc:
    def __init__(self, func):
        self.func = func
        self.count = 0  # Initialize the count to 0


    def __call__(self, *args):
        self.count +=1 
        return self.func(*args)


5. Decorate a simple function for computing [Fibonacci numbers](https://en.wikipedia.org/wiki/Fibonacci_sequence)
   ```python
      def fibonacci(n):
          return n if n < 2 else fibonacci(n-2) + fibonacci(n-1)
   ```
   with your decorator `countedFunc`, then call `fibonacci(20)` and check that the value of `fibonacci.count` is $21891$.

In [ ]:
# Fibonacci function
@countedFunc
def fibonacci(n):
    return n if n < 2 else fibonacci(n-2) + fibonacci(n-1)

In [ ]:
fibonacci(20)

In [ ]:
fibonacci.count